In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
filename = input('Enter file name to train on : ')
with open(filename+".txt") as f:
    shakespeare_text = f.read()
print("Total character count : ", len(shakespeare_text))
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

Enter file name to train on : shakespeare_sonnets
Total character count :  94275


In [3]:
checkpoints_filepath = os.path.join(os.curdir, f"checkpoints_{filename}")
directory = os.listdir(checkpoints_filepath)
print(directory)

base_model = directory[int(input('Enter position of model : '))]
root_filepath = os.path.join(os.curdir, f"checkpoints_{filename}\\{base_model}")
file_name = os.listdir(root_filepath)[-1]
load_filepath = f"{root_filepath}\\{file_name}"
print("Loading path : ", load_filepath)

['1624468336', '1624523012', '1624534639', '1624603344']
Enter position of model : 3
Loading path :  .\checkpoints_shakespeare_sonnets\1624603344\1624692026.h5


In [4]:
model = keras.models.load_model(load_filepath)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 512)         847872    
_________________________________________________________________
gru_1 (GRU)                  (None, None, 256)         591360    
_________________________________________________________________
gru_2 (GRU)                  (None, None, 128)         148224    
_________________________________________________________________
gru_3 (GRU)                  (None, None, 64)          37248     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 38)          2470      
Total params: 1,627,174
Trainable params: 1,627,174
Non-trainable params: 0
_________________________________________________________________


In [5]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X, len(tokenizer.word_index))

In [6]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [7]:
def complete_text(text, n_chars=100, temperature=1):
    print(text, end="")
    for _ in range(n_chars):
        next_ch = next_char(text, temperature)
        print(next_ch, end="")
        text += next_ch

In [8]:
input_text = input('Enter some intial text as context : ')
print()
complete_text(input_text)

Enter some intial text as context : Love

Love's breath,
that in the east thou belied, but perjured i,
to say the reven's canst not born of less,
